In [1]:
# %pip install --force-reinstall langchain

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

from langchain.chains import LLMChain
from langchain.prompts import (
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain.memory import ConversationSummaryMemory, FileChatMessageHistory, ConversationBufferMemory

In [3]:
chat = ChatOpenAI(base_url="http://172.21.240.1:1234/v1", 
             api_key="not-needed")

llm = OpenAI(base_url="http://172.21.240.1:1234/v1", 
             api_key="not-needed",
             temperature =0
             )

In [4]:


# memory = ConversationBufferMemory(
#     chat_memory= FileChatMessageHistory( 'messages.json' ),
#     memory_key="messages", 
#     return_messages=True,  # returns message classes
# )

memory = ConversationSummaryMemory(
    memory_key="messages", 
    return_messages=True,  # returns message classes
    llm = chat
)

In [5]:
memory.save_context({"input": "hi"}, {"output": "whats up"})

In [6]:
memory.load_memory_variables({})

{'messages': [SystemMessage(content="Hi, what's up?")]}

In [7]:
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}"),
    ],
)

In [8]:
chain = LLMChain(llm=chat,
                 prompt=prompt,
                 memory=memory,
                 verbose = True)

In [9]:
is_loop = True
while is_loop == True:
    content = input(">> ")

    if not content:
        is_loop = False
        break

    result = chain({"content": content})

    print(result)
    print(result["text"])



> Entering new LLMChain chain...
Prompt after formatting:
System: Hi, what's up?
Human: whats 1 + 1

> Finished chain.
{'content': 'whats 1 + 1', 'messages': [SystemMessage(content="Hi, what's up?")], 'text': '2'}
2


> Entering new LLMChain chain...
Prompt after formatting:
System: Hi, what's up? The AI responds with "2" when asked what 1 + 1 is.
Human: ok.  can you add 5 to that?

> Finished chain.
{'content': 'ok.  can you add 5 to that?', 'messages': [SystemMessage(content='Hi, what\'s up? The AI responds with "2" when asked what 1 + 1 is.')], 'text': '5 + 2 = 7'}
5 + 2 = 7


> Entering new LLMChain chain...
Prompt after formatting:
System: Hi, what's up? The AI responds with "2" when asked what 1 + 1 is. The human asks if the AI can add 5 to that. The AI responds with "5 + 2 = 7".
Human: you're crushing it!  add 9 more please

> Finished chain.
{'content': "you're crushing it!  add 9 more please", 'messages': [SystemMessage(content='Hi, what\'s up? The AI responds with "2" when 